# Phase 2 — DSL Design

This notebook defines the Domain Specific Language (DSL) for expressing trading rules.

The DSL must support:
- ENTRY and EXIT sections
- Boolean logic: AND, OR
- Comparisons: > < >= <= ==
- Indicators: SMA(series, N), RSI(series, N)
- Cross events: series crosses above/below series
- Parentheses for grouping

In [1]:
import os

# Make sure the folder exists
os.makedirs("dsl", exist_ok=True)

In [6]:
dsl_grammar = r"""
// ----------------------------
// DSL Grammar for Trading Rules
// ----------------------------

?start: section+

// Sections
section: "ENTRY:" expr             -> entry_section
       | "EXIT:" expr              -> exit_section

// Expression precedence
?expr: or_expr
?or_expr: and_expr
        | or_expr "OR" and_expr    -> or_op

?and_expr: term
         | and_expr "AND" term     -> and_op

// Groups & atomic terms
?term: comparison
     | cross
     | "(" expr ")"

// Comparison operations
comparison: operand COMP operand   -> comparison

// Cross events
cross: operand "crosses" "above" operand  -> cross_above
     | operand "crosses" "below" operand  -> cross_below

// Valid operands
operand: INDICATOR                 -> indicator
       | IDENT                     -> ident
       | NUMBER                    -> number
       | "yesterday_high"          -> yest_high

// Allowed tokens
INDICATOR: /(SMA|sma|RSI|rsi)\([a-zA-Z0-9_,\s]+\)/
IDENT: /(open|high|low|close|volume)/i
NUMBER: /\d+(\.\d+)?/

COMP: ">" | "<" | ">=" | "<=" | "=="

// Ignore whitespace and comments
%ignore /[ \t\f\r]+/
%ignore /[\n]+/
%ignore /\#.*/ 
"""


In [7]:
dsl_spec = {
    "sections": ["ENTRY", "EXIT"],
    "operators": [">", "<", ">=", "<=", "=="],
    "boolean": ["AND", "OR"],
    "series": ["open", "high", "low", "close", "volume"],
    "indicators": {
        "SMA": {"params": ["series", "int"]},
        "RSI": {"params": ["series", "int"]},
    },
    "cross_events": ["crosses above", "crosses below"],
    "special_series": ["yesterday_high"],
    "numbers": "int or float",
    "grouping": "parentheses supported",
}


In [8]:
import json
print(json.dumps(dsl_spec, indent=4))

{
    "sections": [
        "ENTRY",
        "EXIT"
    ],
    "operators": [
        ">",
        "<",
        ">=",
        "<=",
        "=="
    ],
    "boolean": [
        "AND",
        "OR"
    ],
    "series": [
        "open",
        "high",
        "low",
        "close",
        "volume"
    ],
    "indicators": {
        "SMA": {
            "params": [
                "series",
                "int"
            ]
        },
        "RSI": {
            "params": [
                "series",
                "int"
            ]
        }
    },
    "cross_events": [
        "crosses above",
        "crosses below"
    ],
    "special_series": [
        "yesterday_high"
    ],
    "numbers": "int or float",
    "grouping": "parentheses supported"
}


In [9]:
import os

# Making sure the folder exists
os.makedirs("dsl", exist_ok=True)

In [10]:
dsl_examples = [
    """
    ENTRY:
    close > SMA(close,20) AND volume > 1000000

    EXIT:
    RSI(close,14) < 30
    """,

    """
    ENTRY:
    close crosses above yesterday_high

    EXIT:
    close < SMA(close,10)
    """,

    """
    ENTRY:
    (close > SMA(close,50) AND volume > 2000000)
    OR
    (close > SMA(close,20) AND RSI(close,14) < 40)

    EXIT:
    RSI(close,14) < 25
    """
]

for ex in dsl_examples:
    print("------ DSL Example ------")
    print(ex)


------ DSL Example ------

    ENTRY:
    close > SMA(close,20) AND volume > 1000000

    EXIT:
    RSI(close,14) < 30
    
------ DSL Example ------

    ENTRY:
    close crosses above yesterday_high

    EXIT:
    close < SMA(close,10)
    
------ DSL Example ------

    ENTRY:
    (close > SMA(close,50) AND volume > 2000000)
    OR
    (close > SMA(close,20) AND RSI(close,14) < 40)

    EXIT:
    RSI(close,14) < 25
    


In [5]:
with open("dsl/grammar.lark", "w") as f:
    f.write(dsl_grammar)